In [28]:
%matplotlib inline 


import random
import math
from collections import defaultdict
from operator import itemgetter

import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

# uid diaryid
data = pd.read_table('data/user_diary.txt', sep=',', names=['uid', 'diary_id'])[:2000]


def splitData(data, M, k, seed):
    test = defaultdict(tuple)  # list of [uid, diary_id]
    train = defaultdict(tuple)
    random.seed(seed)
    for row in data.itertuples():
        if random.randint(0, M) == k:
            test[row.uid] += (row.diary_id,)
        else:
            train[row.uid] += (row.diary_id,)
    return train, test

train, test = splitData(data, 10, 5, 2)


def Precision(train, test, W):
    hit = 0
    all = 0
    
    for uid in train.keys():
        tu = test[uid]
        
        rank = GetRecommendation(uid, train, W, 10)
        for item, pui in rank.items():
            if item in tu:
                 hit += 1
        all += len(rank)
    return hit / (all * 1.0)


def GetRecommendation(uid, train, W, K=50):
    rank = defaultdict(tuple)
    interacted_items = train[uid]
    for v, wuv in sorted(W[uid].items(), key=itemgetter(1), reverse=True)[0:K]:
        for i, rvi in train.items():
            if i in interacted_items:
                continue
            rank[i] += rvi
    return rank


def UserSimilarity(train):
    item_users = defaultdict(tuple)
    for uid, diary_ids in train.items():
        for diary_id in diary_ids:
            item_users[diary_id] += (uid,)

    C = defaultdict(lambda: defaultdict(int))
    N = defaultdict(int)
    for diary_id, uids in item_users.items():
        for u in uids:
            N[u] += 1
            
            for v in uids:
                if u == v: continue
                C[u][v] += 1

    #calculate finial similarity matrix W
    W = defaultdict(lambda: defaultdict(int))
    for u, related_users in C.items():
        for v, cuv in related_users.items():
            W[u][v] = cuv / math.sqrt(N[u] * N[v])

    return W


W = UserSimilarity(train)
Precision(train, test, W)

0.0